# Telco Customer Churn Prediction Assignment Problem Statement - 7

#### Name: Yenduru Poojith Manideep
#### BITS ID: 2024ac05966@wilp.bits-pilani.ac.in

#### Objective: Predict customer churn for a telecommunications company using machine learning models

## Section 1: Import Libraries and Dataset

In [ ]:
# Importing Required Libraries
import pandas as pd  # For data manipulation and analysis
import numpy as np   # For numerical operations
import matplotlib.pyplot as plt  # For plotting
import seaborn as sns  # For advanced visualization
import warnings
warnings.filterwarnings('ignore')

# Setting visualization style
sns.set(style='whitegrid', palette='muted', font_scale=1.1)

### Load the Dataset


In [ ]:
# Loading Telco Customer Churn Dataset
df = pd.read_csv('Telco-Dataset.csv')
print('Dataset Loaded. Shape of the Dataset is:', df.shape)

## Section 2: Data Visualization and Exploration

In [ ]:
df.head(10)  # Show 10 rows for a richer preview

In [ ]:
# Dataset metadata & null scan
print('Dataset shape:', df.shape)

print('\nColumn Types and Null Scan:')
print(df.info())

In [ ]:

# df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')
# df['TotalCharges'] = df['TotalCharges'].fillna(0)

# print('\nStatistical Description (Numerical Columns):')
# display(df.describe())

In [ ]:
# # Churn value counts to check balance
# plt.figure(figsize=(4,3))
# sns.countplot(data=df, x='Churn')
# plt.title('Churn Class Distribution')
# plt.show()

# # Distribution of numerical features
# num_cols = ['tenure', 'MonthlyCharges', 'TotalCharges']
# fig, axs = plt.subplots(1, 3, figsize=(18, 4))
# for i, col in enumerate(num_cols):
#     sns.histplot(df[col], bins=25, kde=True, ax=axs[i])
#     axs[i].set_title(f'Distribution of {col}')
# plt.tight_layout()
# plt.show()

# # Distribution of categorical features
# cat_cols = ['gender', 'SeniorCitizen', 'Partner', 'Dependents', 'PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity',
#             'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod']

# plt.figure(figsize=(16, 16))
# for i, col in enumerate(cat_cols, 1):
#     plt.subplot(4, 4, i)
#     sns.countplot(x=col, data=df, color='steelblue')
#     plt.tight_layout()
# plt.suptitle('Categorical Feature Distributions', y=1.02, fontsize=16)
# plt.show()

In [ ]:
# Correlational matrix (numerical)
# plt.figure(figsize=(6,4))
# corr = df[['tenure', 'MonthlyCharges']].corr(method='pearson')
# sns.heatmap(corr, annot=True, cmap='coolwarm', fmt='.2f')
# plt.title('Correlation Matrix (Numerical Features)')
# plt.show()

In [ ]:

df = df.replace(['', ' ', 'NA', 'na', 'NaN', 'null', 'NULL'], np.nan)


# Scan for missing values and datatypes
print('Missing values per column:')
print(df.isnull().sum())

df['TotalCharges'] = df['TotalCharges'].replace(' ', pd.NA)
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')

print(f"Missing TotalCharges before better imputation: {df['TotalCharges'].isna().sum()}")


# # Better imputation logic for missing values in TotalCharges

# # For 0 tenure customers, set TotalCharges to MonthlyCharges
zero_tenure_mask = (df['tenure'] == 0) & df['TotalCharges'].isna()
df.loc[zero_tenure_mask, 'TotalCharges'] = df.loc[zero_tenure_mask, 'MonthlyCharges']

# # For other missing values, estimate using tenure * MonthlyCharges  
missing_mask = df['TotalCharges'].isna()
df.loc[missing_mask, 'TotalCharges'] = df.loc[missing_mask, 'tenure'] * df.loc[missing_mask, 'MonthlyCharges']

print(f"Missing TotalCharges after better imputation: {df['TotalCharges'].isna().sum()}")


In [ ]:
# Visual inspection for outliers
fig, axs = plt.subplots(1, 3, figsize=(18,4))
for i, col in enumerate(['tenure', 'MonthlyCharges', 'TotalCharges']):
    sns.boxplot(x=df[col], ax=axs[i], color='lightcoral')
    axs[i].set_title(f'Boxplot - {col}')
plt.tight_layout()
plt.show()

# Skewness check
skew_vals = df[['tenure','MonthlyCharges', 'TotalCharges']].skew()
print('Skewness of Numeric Features:')
print(skew_vals)

# Why not immediately remove outliers? For customer churn, outliers are likely legitimate (very high tenure/charges are loyal customers), so removal may lose valuable business insight. We'll scale these instead rather than cap/drop.

In [ ]:
# Drop customerID
df_ml = df.drop('customerID', axis=1)

#Skewness of TotalCharges is  0.963316 - We need to reduce the skewness

from sklearn.preprocessing import PowerTransformer as pt

pt = pt(method="yeo-johnson")
df_ml['TotalCharges'] = pt.fit_transform(df_ml[["TotalCharges"]])

# Verify skewness after transformation
print('Skewness after yeo-johnson on TotalCharges:', df_ml['TotalCharges'].skew())

# Optional: Re-plot histogram for TotalCharges to visualize improvement
plt.figure(figsize=(6,4))
sns.histplot(df_ml['TotalCharges'], kde=True)
plt.title('Distribution of TotalCharges after log1p')
plt.show()


In [ ]:
# Recode redundant categories for internet-dependent features
internet_dependent = ['OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies']
for col in internet_dependent:
    df_ml[col] = df_ml[col].replace('No internet service', 'No')

# Similarly for MultipleLines
df_ml['MultipleLines'] = df_ml['MultipleLines'].replace('No phone service', 'No')

In [ ]:
from sklearn.preprocessing import StandardScaler
# Instantiate scaler
scaler = StandardScaler()
num_cols = ['tenure','MonthlyCharges','TotalCharges']
df_ml[num_cols] = scaler.fit_transform(df_ml[num_cols])

# Verify scaling
df_ml[num_cols].describe().T[['mean','std']]

In [ ]:
df_ml.head(10)  # Show 10 rows for a richer preview

In [ ]:
## Feature Engineering -> Creating new Features

# Create new predictive features
df_ml['AvgMonthlySpend'] = df_ml['TotalCharges'] / (df_ml['tenure'] + 1)
df_ml['IsMonthToMonth'] = (df_ml['Contract'] == 'Month-to-month').astype(int)
df_ml['IsElectronicCheck'] = (df_ml['PaymentMethod'] == 'Electronic check').astype(int)
df_ml['HasPartnerOrDependents'] = ((df_ml['Partner'] == 'Yes') | (df_ml['Dependents'] == 'Yes')).astype(int)
df_ml['HasPhoneAndInternet'] = ((df_ml['PhoneService'] == 'Yes') & (df_ml['InternetService'] != 'No')).astype(int)

# Count of additional services
service_cols = ['OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies']
df_ml['AdditionalServices'] = sum([df_ml[col].map({'Yes': 1, 'No': 0}) for col in service_cols])

# df_ml["tenure_group"] = pd.cut(
#     df_ml["tenure"],
#     bins=[0, 12, 24, 48, 60, 72],
#     labels=["0-12", "12-24", "24-48", "48-60", "60-72"],
# )
df_ml["is_long_contract"] = df_ml["Contract"].isin(["One year", "Two year"]).astype(int)
df_ml["fiber_streaming"] = (
    (df_ml["InternetService"] == "Fiber optic") & (df["StreamingTV"] == "Yes")
).astype(int)

In [ ]:
from sklearn.preprocessing import StandardScaler
# Instantiate scaler
scaler = StandardScaler()
num_cols = ['tenure','MonthlyCharges','TotalCharges', 'AvgMonthlySpend', 'IsMonthToMonth', 'AdditionalServices']
df_ml[num_cols] = scaler.fit_transform(df_ml[num_cols])

# Verify scaling
df_ml[num_cols].describe().T[['mean','std']]

In [ ]:

# Encode churn label
df_ml['Churn'] = df_ml['Churn'].map({'Yes': 1, 'No': 0})

# List categorical features (excluding target)
cat_feats = [col for col in df_ml.columns if df_ml[col].dtype == 'object' and col != 'Churn']

# Show unique values for categorical features (documentation)
for col in cat_feats:
    print(f"{col}: {df_ml[col].unique()}")

# Apply one-hot encoding (assign drop_first=False so all categories explicit)
df_ml = pd.get_dummies(df_ml, columns=cat_feats, drop_first=False)

print('Shape after encoding:', df_ml.shape)
df_ml.head()

In [ ]:
from sklearn.feature_selection import mutual_info_classif

X = df_ml.drop('Churn', axis=1)
y = df_ml['Churn']
mi_scores = mutual_info_classif(X, y, discrete_features='auto', random_state=42)

feat_importance = pd.Series(mi_scores, X.columns).sort_values(ascending=False)

# Top 15 features
feat_importance[:15].plot(kind='barh', figsize=(8,6))
plt.gca().invert_yaxis()
plt.title('Feature Importance (Mutual Information w/ Churn)')
plt.show()

# Display top features (table)
display(feat_importance.head(15).to_frame('MI Score'))

# Drop low-importance features (MI <= 0.01, keep important only)
selected_feats = feat_importance[feat_importance > 0.01].index.tolist()
if 'Churn' not in selected_feats:
    selected_feats.append('Churn')
df_ml = df_ml[selected_feats]


In [ ]:
from sklearn.model_selection import train_test_split
# Data for modeling
X = df_ml.drop('Churn', axis=1)
y = df_ml['Churn']

# 80/20 split (default)
X_train_80, X_test_20, y_train_80, y_test_20 = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
print(f"80/20 split - Train: {X_train_80.shape}, Test: {X_test_20.shape}")

# 70/30 split
X_train_70, X_test_30, y_train_70, y_test_30 = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)
print(f"70/30 split - Train: {X_train_70.shape}, Test: {X_test_30.shape}")

# 90/10 split
X_train_90, X_test_10, y_train_90, y_test_10 = train_test_split(X, y, test_size=0.10, random_state=42, stratify=y)
print(f"90/10 split - Train: {X_train_90.shape}, Test: {X_test_10.shape}")

In [ ]:
print('Churn distribution in full set:', y.mean().round(3))
print('Churn in 80% train:', y_train_80.mean().round(3))
print('Churn in 20% test:', y_test_20.mean().round(3))
# Check other splits
print('Churn in 70% train:', y_train_70.mean().round(3))
print('Churn in 30% test:', y_test_30.mean().round(3))
print('Churn in 90% train:', y_train_90.mean().round(3))
print('Churn in 10% test:', y_test_10.mean().round(3))

# Apply SMOTE to address class imbalance on training sets only
# from imblearn.over_sampling import SMOTE
# smote = SMOTE(random_state=42, k_neighbors=3)

from imblearn.combine import SMOTEENN
smote = SMOTEENN(random_state=42)

X_train_80_bal, y_train_80_bal = smote.fit_resample(X_train_80, y_train_80)
X_train_70_bal, y_train_70_bal = smote.fit_resample(X_train_70, y_train_70)
X_train_90_bal, y_train_90_bal = smote.fit_resample(X_train_90, y_train_90)

# X_train_80_bal, y_train_80_bal = X_train_80, y_train_80
# X_train_70_bal, y_train_70_bal = X_train_70, y_train_70
# X_train_90_bal, y_train_90_bal = X_train_90, y_train_90


# Verify balance
print('Balanced 80/20 train:', y_train_80_bal.mean().round(3))
print('Balanced 70/30 train:', y_train_70_bal.mean().round(3))
print('Balanced 90/10 train:', y_train_90_bal.mean().round(3))

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

logreg = LogisticRegression(solver='liblinear', max_iter=1000, class_weight='balanced')
# Tune penalty and regularization strength
param_grid_lr = {
    'penalty': ['l1', 'l2'], 
    'C': [0.1, 0.5, 1, 2,3],
    'class_weight': ['balanced', None]
}
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Grid search for optimal hyperparameters on 80/20 split
gs_lr = GridSearchCV(logreg, param_grid_lr, cv=cv, scoring='f1', n_jobs=-1)
gs_lr.fit(X_train_80_bal, y_train_80_bal)
print(f"Best LR params: {gs_lr.best_params_}; best CV ROC AUC: {gs_lr.best_score_:.3f}")

# Predict on test
y_pred_lr = gs_lr.predict(X_test_20)
y_proba_lr = gs_lr.predict_proba(X_test_20)[:,1]

print("Accuracy:", accuracy_score(y_test_20, y_pred_lr))
print("Precision:", precision_score(y_test_20, y_pred_lr))
print("Recall:", recall_score(y_test_20, y_pred_lr))
print("F1 Score:", f1_score(y_test_20, y_pred_lr))
print("ROC AUC:", roc_auc_score(y_test_20, y_proba_lr))


In [ ]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state=42, class_weight='balanced')
param_grid_dt = {
    'max_depth': [5, 10, 15, 20],
    'min_samples_split': [2, 5, 10, 20],
    'min_samples_leaf': [1, 2, 5],
    'class_weight': ['balanced', None]
}
gs_dt = GridSearchCV(dt, param_grid_dt, cv=cv, scoring='f1', n_jobs=-1)
gs_dt.fit(X_train_80_bal, y_train_80_bal)
print(f"Best Decision Tree params: {gs_dt.best_params_}; best CV ROC AUC: {gs_dt.best_score_:.3f}")
y_pred_dt = gs_dt.predict(X_test_20)
y_proba_dt = gs_dt.predict_proba(X_test_20)[:,1]

print("Accuracy:", accuracy_score(y_test_20, y_pred_dt))
print("Precision:", precision_score(y_test_20, y_pred_dt))
print("Recall:", recall_score(y_test_20, y_pred_dt))
print("F1 Score:", f1_score(y_test_20, y_pred_dt))
print("ROC AUC:", roc_auc_score(y_test_20, y_proba_dt))

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier()
param_grid_knn = {
    'n_neighbors': [3, 5, 7, 11, 15],
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan']
}
gs_knn = GridSearchCV(knn, param_grid_knn, cv=cv, scoring='f1', n_jobs=-1)
gs_knn.fit(X_train_80_bal, y_train_80_bal)
print(f"Best KNN params: {gs_knn.best_params_}; best CV ROC AUC: {gs_knn.best_score_:.3f}")
y_pred_knn = gs_knn.predict(X_test_20)
y_proba_knn = gs_knn.predict_proba(X_test_20)[:,1]

print("Accuracy:", accuracy_score(y_test_20, y_pred_knn))
print("Precision:", precision_score(y_test_20, y_pred_knn))
print("Recall:", recall_score(y_test_20, y_pred_knn))
print("F1 Score:", f1_score(y_test_20, y_pred_knn))
print("ROC AUC:", roc_auc_score(y_test_20, y_proba_knn))

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(random_state=42, class_weight='balanced')
param_grid_rf = {
    'n_estimators': [100, 200],
    'max_depth': [10, 15, None],
    'min_samples_split': [2, 5, 10],
    'class_weight': ['balanced', None]
}
gs_rf = GridSearchCV(rf, param_grid_rf, cv=cv, scoring='f1', n_jobs=-1)
gs_rf.fit(X_train_80_bal, y_train_80_bal)
print(f"Best RF params: {gs_rf.best_params_}; best CV ROC AUC: {gs_rf.best_score_:.3f}")
y_pred_rf = gs_rf.predict(X_test_20)
y_proba_rf = gs_rf.predict_proba(X_test_20)[:,1]

print("Accuracy:", accuracy_score(y_test_20, y_pred_rf))
print("Precision:", precision_score(y_test_20, y_pred_rf))
print("Recall:", recall_score(y_test_20, y_pred_rf))
print("F1 Score:", f1_score(y_test_20, y_pred_rf))
print("ROC AUC:", roc_auc_score(y_test_20, y_proba_rf))

In [ ]:
# Calculate all metrics for test set (80/20)
from sklearn.metrics import classification_report, roc_auc_score, roc_curve, confusion_matrix

def eval_metrics(y_true, y_pred, y_proba, name):
    return {
        'Model': name,
        'Accuracy': accuracy_score(y_true, y_pred),
        'Precision': precision_score(y_true, y_pred),
        'Recall': recall_score(y_true, y_pred),
        'F1': f1_score(y_true, y_pred),
        'ROC_AUC': roc_auc_score(y_true, y_proba)
    }

results = []
results.append(eval_metrics(y_test_20, y_pred_lr, y_proba_lr, 'Logistic Regression'))
results.append(eval_metrics(y_test_20, y_pred_dt, y_proba_dt, 'Decision Tree'))
results.append(eval_metrics(y_test_20, y_pred_knn, y_proba_knn, 'KNN'))
results.append(eval_metrics(y_test_20, y_pred_rf, y_proba_rf, 'Random Forest'))

import pandas as pd
perf_df = pd.DataFrame(results).set_index('Model')
display(perf_df)

In [ ]:
# Plot performance metrics for all models
plt.figure(figsize=(10,6))
perf_df[['Accuracy','Precision','Recall','F1','ROC_AUC']].plot(kind='bar', ax=plt.gca(), rot=15)
plt.title('Comparison of Model Performance Metrics')
plt.ylabel('Score')
plt.ylim(0, 1)
plt.legend(loc='lower right')
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(8,6))
models_pred = [
    ('Logistic Regression', y_proba_lr),
    ('Decision Tree', y_proba_dt),
    ('KNN', y_proba_knn),
    ('Random Forest', y_proba_rf)
]
for name, yproba in models_pred:
    fpr, tpr, _ = roc_curve(y_test_20, yproba)
    plt.plot(fpr, tpr, label=f'{name} (AUC: {roc_auc_score(y_test_20, yproba):.2f})')
plt.plot([0,1],[0,1],'k--', lw=0.8)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curves')
plt.legend()
plt.tight_layout()
plt.show()